# Analisis Exploratorio - Datos UNIAJC

Fuentes de datos analizadas

* DatosI
* DatosII
* DatosIII

## Preparación entorno

In [3]:
#Importación librerias a implementar
import pandas as pd

In [4]:
#Class para dar forma a los print
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

In [5]:
""" Función para calcular la edad
calcular_edad(fecha de nacimiento):
return edad"""

def calcular_edad(fecha_nacimiento):
    from datetime import date
    hoy = date.today()
    edad = int(hoy.year - fecha_nacimiento.year)
    
    return edad

## Conexión fuentes de datos

Iniciamos con la conexión a la base de datos de estudiantes de la Universiad Antonio Jose Camacho, que se encuentra en un único archivo de excel

In [ ]:
#Se establece el archivo xlsx a cargar, luego de haber sido unificado
camacho = pd.ExcelFile('CamachoUnido_limpiov1.xlsx')

In [ ]:
#Se carga cada uno de las hojas del excel, las cuales representan semestres diferentes
df_20191 = pd.read_excel(camacho,'DatosI_Sistemas')
df_20192 = pd.read_excel(camacho,'DatosII_Sistemas')
df_20201 = pd.read_excel(camacho,'DatosIII_Sistemas')

### DatosI - 20191
Exploramos el conjunto de datos de los estudiantes matriculados en el semestre 20191 en el programa de Ingeniería de Sistemas (no se incluye homologación)

In [ ]:
#Se conocen los tipos de dato de cada variable
df_20191.info()

In [ ]:
#Se conocen valores faltantes por columna
print('Valores faltantes en cada columna', df_20191.isnull().sum())

Se obtienes algunos números importantes del dataset, para conocerlo mejor

In [ ]:
#Cantidad de estudiantes únicos
id_df20191 = df_20191['ID']
id_df20191 = id_df20191.drop_duplicates()
print(color.BOLD +'Cantidad de estudiantes en 20191: '+ color.END,id_df20191.count())

In [ ]:
#cantidad de estudiantes sin estrato
mask_estrato = df_20191.ESTRATO.isnull()
id_df20191_sin_estrato = df_20191[mask_estrato]['ID']
id_df20191_sin_estrato = id_df20191_sin_estrato.drop_duplicates()
print(color.BOLD +'Cantidad de estudiantes sin estrato definido: '+ color.END,id_df20191_sin_estrato.count())
print(color.BOLD +'ID estudiantes sin estrato definido: \n'+ color.END,id_df20191_sin_estrato)

In [ ]:
#cantidad de estudiantes sin país de nacimiento, departamento de nacimiento y municipio de nacimiento
mask_lugar_nacimiento = df_20191.PAIS_NACIMIENTO.isnull()
id_df20191_sin_lugar_nacimiento = df_20191[mask_lugar_nacimiento]['ID']
id_df20191_sin_lugar_nacimiento = id_df20191_sin_lugar_nacimiento.drop_duplicates()
print(color.BOLD +'ID estudiantes sin lugar de nacimiento definido: \n'+ color.END,id_df20191_sin_lugar_nacimiento)
print(color.BOLD +'Cantidad de estudiantes sin lugar de nacimiento: '+ color.END,id_df20191_sin_lugar_nacimiento.count())

In [ ]:
#cantidad de estudiantes sin país de nacimiento, departamento de nacimiento y municipio de nacimiento
mask_lugar_residencia = df_20191.PAIS_RESIDENCIA.isnull()
id_df20191_sin_lugar_residencia = df_20191[mask_lugar_residencia]['ID']
id_df20191_sin_lugar_residencia = id_df20191_sin_lugar_residencia.drop_duplicates()
print(color.BOLD +'ID estudiantes sin lugar de residencia definido: \n'+ color.END,id_df20191_sin_lugar_residencia)
print(color.BOLD +'Cantidad de estudiantes sin lugar de residencia: '+ color.END,id_df20191_sin_lugar_nacimiento.count())
  

In [ ]:
#faltantes_20191 = {'Estrato': id_df20191_sin_estrato.values.tolist(),'Lugar Nacimiento': id_df20191_sin_lugar_nacimiento.values.tolist()}
#print(faltantes_20191)

### Tratamiento de valores faltantes

In [ ]:
#En el Dataset en las columnas COLEGIO y COLEGIO2 tienen distribuido el nombre del colegio donde los estudiantes cursaron su educación básica secundaria
#Se rellenan los null de la columna Colegio por el valor en Colegio2
colegio = df_20191.loc[:,['ID','COLEGIO','COLEGIO2']]
for i in range(colegio.shape[0]):
    if colegio.COLEGIO[i] == 'Sin Definir':
        colegio.COLEGIO[i] = colegio.COLEGIO2[i]
colegio = colegio.drop(columns = 'COLEGIO2').drop_duplicates()
#colegio = colegio.drop_duplicates()

In [ ]:
#Se transforma la variable de categórica a numérica
colegio_dummies = colegio
colegio_dummies['COLEGIO'] = pd.get_dummies(colegio.COLEGIO)

In [ ]:
#Se unifican los métodos de pago
""""metodo_pago = df_20191.loc[:,['ID','FINAN ICETEX','FINAN BANCARIA','PAGO DIRECTO']]
metodo_pago = metodo_pago.drop_duplicates()
metodo_pago = metodo_pago.fillna('Otro')

for i in range(metodo_pago.shape[0]):
    if metodo_pago.loc[i,'FINAN ICETEX'] == 'Otro':
        if metodo_pago.loc[i,'FINAN BANCARIA'] == 'Otro':
            metodo_pago.loc[i,'METODO_PAGO'] = 'PAGO DIRECTO'
        else:
            metodo_pago.loc[i,'METODO_PAGO'] = 'FINAN BANCARIA'
    else:
        metodo_pago.loc[i,'METODO_PAGO'] = 'FINAN ICETEX'
metodo_pago = metodo_pago.drop(columns = ['FINAN ICETEX','FINAN BANCARIA','PAGO DIRECTO']).drop_duplicates()


#Verificación que no hayan valores nulos en metodo_pago
metodo_pago.isnull().sum()"""

In [ ]:
#Calculo el cantidad de materias
cant_materia = pd.DataFrame(columns=['ID','COD_MATERIA'])
row = 0
for i in id_df20191:
    mask = df_20191['ID'] == i
    cant_materia.loc[row,'ID'] = i
    cant_materia.loc[row,'CANT_MATERIAS'] = len(df_20191[mask]['COD_MATERIA'].unique())
    row = row +1
cant_materia = cant_materia.drop(columns = ['COD_MATERIA'])

In [ ]:
#Se verifica que el cálculo se haya realizado correctamente
cant_materia

In [ ]:
#Calculo de edad de cada estudiante
df_edad = df_20191.loc[:,['ID','FECHA_NACIMIENTO']]
df_edad = df_edad.drop_duplicates()
edades = pd.DataFrame(columns = ['ID','EDAD'])
row = 0
for i in df_edad['FECHA_NACIMIENTO']:
    edades.loc[row,'EDAD'] = int(calcular_edad(i))
    row = row + 1
row = 0    
for i in df_edad['ID']:
    edades.loc[row,'ID'] = i
    row = row + 1

In [ ]:
#Calculo de edad de cada estudiante
estrato = df_20191.loc[:,['ID','ESTRATO']].drop_duplicates().fillna(0)

In [ ]:
genero = df_20191.loc[:,['ID','GENERO']].drop_duplicates()
genero_dummies = pd.get_dummies(genero.GENERO)

In [ ]:
#Se eliminan columnas que no se utilizarán en la primera versión del análisis, los días de la semana donde se recibia clase. Ni fechas de ejecución de los pagos
df_20191 = df_20191.drop(columns=['GENERO','ESTADO_CIVIL','PAIS_NACIMIENTO','DPTO_NACIMIENTO','CIUDAD_NACIMIENTO','DPTO_RESIDENCIA','CIUDAD_GENERAL','BARRIO','ESTRATO','DIRECCION','PAIS_RESIDENCIA','FECHA_NACIMIENTO','LUNES','MARTES','MIERCOLES','JUEVES','VIERNES','SABADO','DOMINGO','COLEGIO','COLEGIO2','COD_PROG','PROGRAMA','MATERIA','COD_MATERIA','GRUPO','EVAC_ID','EVALUACION','CALF_PARCIAL','FECHA_MATRICULA','FECHA DE PAGO','SNP','DOCENTE','ESTADO','PER_ADMISION','REAC_TIPO','GRUP_ID','FINAN ICETEX','FINAN BANCARIA','PAGO DIRECTO']).drop_duplicates()


### Etiqueta de desertor
Para categorizar a un estudiante como desertor aplicaremos el criterio utilizado por el Ministerio de educación, esto es, un estudiante con dos semestres sin matricularse en el programa académico.

Para ello compararemos los ID únicos del semestre 20191 con los del 20201, estos serán estudiantes candidatos a desertores, los cuales verificaremos si tampoco estuvo activo en el semestre inmediatamente anterior, 20192. De cumplirse la ausencia en el semestre 20201 y 20192 este estudiante será considerado Desertor

In [ ]:
#ID 20192
print(color.BOLD + '  ID 20192' + color.END)
print(df_20192['ID'].value_counts())
id_df20192 = df_20192['ID']
id_df20192 = id_df20192.drop_duplicates()
id_df20191 = id_df20191.values
id_df20191 = id_df20191.tolist()
id_df20192 = id_df20192.values
id_df20192 = id_df20192.tolist()

In [ ]:
#ID 20201
print(color.BOLD + '  ID 20201' + color.END)
print(df_20201['ID'].value_counts())
id_df20201 = df_20201['ID']
id_df20201 = id_df20201.drop_duplicates()
id_df20201 = id_df20201.values
id_df20201 = id_df20201.tolist()

In [ ]:
candidatos_desertores = []
for i in range(len(id_df20191)):
    if id_df20191[i] in id_df20201:
        continue
    else:
        candidatos_desertores.append(id_df20191[i])

In [ ]:
print(f"Estudiantes candidatos a desertores: {len(candidatos_desertores)}\n",candidatos_desertores)

In [ ]:
desertores_reales = []
for i in range(len(candidatos_desertores)):
    if candidatos_desertores[i] in id_df20192:
        continue
    else:
        desertores_reales.append(candidatos_desertores[i])

In [ ]:
print(f"Estudiantes desertores: {len(desertores_reales)} \n",desertores_reales)

In [ ]:
#Se añade el estado académico al dataset para cada ID de estudiante
estado_academico = pd.DataFrame(columns=['ID','ESTADO'])
row = 0
for i in id_df20191:
    if i in desertores_reales:
        estado_academico.loc[row,'ESTADO'] = 'Desertor'
    else:
        estado_academico.loc[row,'ESTADO'] = 'Activo'
    estado_academico.loc[row,'ID'] = i
    row = row + 1

In [ ]:
#Se verifica que se haya realizado el proceso
estado_academico

In [ ]:
#Se transforma la variable de categórica a numérica
estado_academico_dummies = pd.get_dummies(estado_academico.ESTADO)

## Formando Dataset1 - Entrenamiento

Se unifican las variables que han sido transformadas de categóricas a numéricas en un único dataset

In [ ]:
""""df_20191['COLEGIO'] = colegio['COLEGIO']
df_20191['CANT_MATERIAS'] = cant_materia['CANT_MATERIAS']
df_20191['METODO_PADO'] = metodo_pago['METODO_PAGO']
df_20191['ESTADO'] = estado_academico['ESTADO']
df_20191['EDAD'] = edades['EDAD']"""

In [ ]:
#final = pd.merge(df_20191,genero_dummies, on = 'ID')
final = pd.merge(df_20191,cant_materia, on = 'ID')
final = pd.merge(final, edades,on = 'ID')
#final = pd.merge(final, estado_academico_dummies,on = 'ID')


In [ ]:
genero_dummies.reset_index(drop=True, inplace=True)
estado_academico_dummies.reset_index(drop=True, inplace=True)
final.reset_index(drop=True, inplace=True)


In [ ]:
final_csv = pd.concat([final,genero_dummies,estado_academico_dummies],axis=1)

In [ ]:
final_csv.isnull().sum()

In [ ]:
final_csv.to_csv('final_normalizado.csv')